# 🧪 Audio Augmentation
This notebook performs various augmentation techniques on audio samples (e.g., time shift, pitch change, noise injection) to increase dataset size and variability for better model generalization.

---

In [ ]:
"""
audio_augmentation.py

This module provides utilities for augmenting audio data using common techniques
such as time stretching, pitch shifting, adding noise, volume adjustment, and cropping.
It also supports saving segments and computing Mel-spectrograms.
"""

import os
import librosa
import numpy as np
import soundfile as sf

# -----------------------------
# Core Audio Augmentation Ops
# -----------------------------

def time_stretch(y: np.ndarray, rate: float = 1.0) -> np.ndarray:
    """Time-stretch the audio signal."""
    if len(y) < 2048:
        return y
    return librosa.effects.time_stretch(y.astype(np.float32), rate=rate)

def pitch_shift(y: np.ndarray, sr: int, n_steps: int = 0) -> np.ndarray:
    """Shift the pitch of the audio signal."""
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def add_noise(y: np.ndarray, noise_level: float = 0.005) -> np.ndarray:
    """Add Gaussian noise to the signal."""
    noise = np.random.randn(len(y))
    return y + noise_level * noise

def adjust_volume(y: np.ndarray, factor: float = 1.0) -> np.ndarray:
    """Adjust the volume of the signal."""
    return y * factor

def random_crop(y: np.ndarray, sr: int, duration: float = 0.1) -> np.ndarray:
    """Crop a random segment of the specified duration."""
    crop_size = int(sr * duration)
    if len(y) <= crop_size:
        return y
    start = np.random.randint(0, len(y) - crop_size)
    return y[start:start + crop_size]

# -----------------------------
# Audio Utility Functions
# -----------------------------

def save_segment(output_path: str, segment: np.ndarray, sr: int) -> None:
    """Save an audio segment to the specified path."""
    sf.write(output_path, segment, sr)
    print(f"Saved {output_path}")

def compute_melspectrogram(segment: np.ndarray, sr: int,
                           n_mels: int = 128, fmax: int = 8000) -> np.ndarray:
    """Compute Mel-spectrogram from an audio segment and flatten it."""
    S = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=n_mels, fmax=fmax)
    return librosa.power_to_db(S, ref=np.max).flatten()

# -----------------------------
# Main Augmentation Pipeline
# -----------------------------

def augment_audio(input_path: str, output_path: str,
                  start_index: int = 0, sr: int = 22050) -> int:
    """
    Augment a single audio file and save outputs.

    Args:
        input_path: Path to the input .wav file
        output_path: Base path prefix for saving files
        start_index: Starting index for naming augmented files
        sr: Sample rate for loading and saving

    Returns:
        Updated index after processing this file
    """
    y, _ = librosa.load(input_path, sr=sr)

    # Save original
    original_path = f"{output_path}_org.wav"
    save_segment(original_path, y, sr)

    # Generate augmented variants
    augmented = [
        time_stretch(y, rate=0.8),           # Slow down
        time_stretch(y, rate=1.2),           # Speed up
        pitch_shift(y, sr, n_steps=2),       # Pitch up
        pitch_shift(y, sr, n_steps=-2),      # Pitch down
        add_noise(y, noise_level=0.01),      # Add noise
        adjust_volume(y, factor=0.5),        # Quieter
        adjust_volume(y, factor=1.5),        # Louder
        random_crop(y, sr, duration=0.1),    # Crop
    ]

    for i, aug in enumerate(augmented):
        aug_index = start_index + i
        aug_path = f"{output_path}_aug_{aug_index}.wav"
        save_segment(aug_path, aug, sr)

    return start_index + len(augmented)

def process_audio_file(input_folder: str, output_folder: str, start_index: int = 0) -> None:
    """
    Process all .wav files in a folder for augmentation.

    Args:
        input_folder: Path to folder with input .wav files
        output_folder: Path to save augmented output
        start_index: Index to begin naming from
    """
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".wav"):
            input_path = os.path.join(input_folder, filename)
            base_name = os.path.splitext(filename)[0]
            output_path = os.path.join(output_folder, base_name)
            start_index = augment_audio(input_path, output_path, start_index)


# Augment bad taps 

In [2]:
resource = '../../resources/material/train-data'
input_folder = f'{resource}/bad-material-taps'
output_folder = f'{resource}/augmented-bad-material-taps'

process_audio_file(input_folder, output_folder)


Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_215_aug_77.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_215_aug_78.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_215_aug_79.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_201_org.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_201_aug_80.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_201_aug_81.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_201_aug_82.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_201_aug_83.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_201_aug_84.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad_tap_201_aug_85.wav
Saved ../../resources/material/train-data/augmented-bad-material-taps/bad

# Augment good taps

In [3]:
resource = '../../resources/material/train-data'
input_folder = f'{resource}/good-material-taps'
output_folder = f'{resource}/augmented-good-material-taps'

process_audio_file(input_folder, output_folder)


Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_org.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_0.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_1.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_2.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_3.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_4.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_5.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_6.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_125_aug_7.wav
Saved ../../resources/material/train-data/augmented-good-material-taps/good_tap_131_org.wav
Saved ../../resources/material/train-data/augmented-good-materia

In [4]:
import os

def count_files_in_folder(folder_path):
    try:
        # List all files and directories in the specified folder
        files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        return len(files)
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")
        return 0

# Example usage
folder_path = '../../resources/material/train-data/augmented-bad-material-taps'  # Replace with your folder path
file_count = count_files_in_folder(folder_path)
print(f"Number of files in '{folder_path}': {file_count}")

Number of files in '../../resources/material/train-data/augmented-bad-material-taps': 1980
